<a href="https://colab.research.google.com/github/Vinicius054/TrabalhoFinalSpark/blob/main/TrabalhoFinalSpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tratando dados de cadastro dos CNPJs do Brasil, retirados da Receita Federal.

In [ ]:
#Instalar as dependencias 
#Instalado Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Instalando  Spark
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
#Extrair Arquivos
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
#Instalando o findspark
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

#Carregamento de Dados

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
     .master('local[*]') \
     .appName('Iniciando com Spark') \
     .config('spark.ui.port', '4050') \
     .getOrCreate()

Faz o dowload da ferramenta Ngrok(Linha 01)
Extrai os arquivos (Linha 02)
Permite que possamos expor nosso localhost na internet pública de forma segura

In [ ]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/nrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

unzip:  cannot find or open ngrok-stable-linux-amd64.zip, ngrok-stable-linux-amd64.zip.zip or ngrok-stable-linux-amd64.zip.ZIP.


In [ ]:
get_ipython().system_raw('./ngrok authtoken ngrok config add-authtoken 2HPehWZIpkqy80elOTfKgUGRpOB_7qswr7xN7uFEx2Nj1AazM')
get_ipython().system_raw('./ngrok http 4050 &')

In [ ]:
!curl -s http://localhost:4040/api/tunnels

In [ ]:
#  spark

#Montagem do Drive

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Carregando os dados da empresa

In [ ]:
import zipfile

In [ ]:
zipfile.ZipFile('/content/drive/MyDrive/curso-spark/empresas.zip','r').extractall('/content/drive/MyDrive/curso-spark')

In [ ]:
path = '/content/drive/MyDrive/curso-spark/empresas'
empresas = spark.read.csv(path, sep =';', inferSchema= True)

In [ ]:
empresas.count()

4585679

#Carregando dados estabelecimento

In [ ]:
import zipfile

In [ ]:
zipfile.ZipFile('/content/drive/MyDrive/curso-spark/estabelecimentos.zip').extractall('/content/drive/MyDrive/curso-spark')
path = '/content/drive/MyDrive/curso-spark/estabelecimentos'
estabelecimentos = spark.read.csv(path, sep = ';', inferSchema = True)

In [ ]:
estabelecimentos.count()

4836219

#Carregando dados socios

In [ ]:
import zipfile
zipfile.ZipFile('/content/drive/MyDrive/curso-spark/socios.zip').extractall('/content/drive/MyDrive/curso-spark')


In [ ]:
path = '/content/drive/MyDrive/curso-spark/socios'
socios = spark.read.csv(path, sep =';', inferSchema= True)

In [ ]:
socios.count()

2046430

#Renomear as colunas do DataFrame

In [ ]:
empresasColNames = ['cnpj_basico', 'razao_social_nome_empresarial', 'natureza_juridica', 'qualificacao_do_responsavel', 'capital_social_da_empresa', 'porte_da_empresa', 'ente_federativo_responsavel']
estabsColNames = ['cnpj_basico', 'cnpj_ordem', 'cnpj_dv', 'identificador_matriz_filial', 'nome_fantasia', 'situacao_cadastral', 'data_situacao_cadastral', 'motivo_situacao_cadastral', 'nome_da_cidade_no_exterior', 'pais', 'data_de_inicio_atividade', 'cnae_fiscal_principal', 'cnae_fiscal_secundaria', 'tipo_de_logradouro', 'logradouro', 'numero', 'complemento', 'bairro', 'cep', 'uf', 'municipio', 'ddd_1', 'telefone_1', 'ddd_2', 'telefone_2', 'ddd_do_fax', 'fax', 'correio_eletronico', 'situacao_especial', 'data_da_situacao_especial']
sociosColNames = ['cnpj_basico', 'identificador_de_socio', 'nome_do_socio_ou_razao_social', 'cnpj_ou_cpf_do_socio', 'qualificacao_do_socio', 'data_de_entrada_sociedade', 'pais', 'representante_legal', 'nome_do_representante', 'qualificacao_do_representante_legal', 'faixa_etaria']

In [ ]:
for index, colName in enumerate(empresasColNames):
        empresas = empresas.withColumnRenamed(f"_c{index}", colName)
empresas.columns


In [ ]:

empresas.limit(5).toPandas()

In [ ]:
empresas.printSchema()

In [ ]:
for index, colName in enumerate(estabsColNames):
        estabelecimentos = estabelecimentos.withColumnRenamed(f"_c{index}", colName)
estabelecimentos.columns

In [ ]:
estabelecimentos.limit(5).toPandas()

In [ ]:
estabelecimentos.printSchema()

In [ ]:
for index, colName in enumerate(sociosColNames):
        socios = socios.withColumnRenamed(f"_c{index}", colName)
socios.columns

In [ ]:
socios.limit(5).toPandas()

In [ ]:
socios.printSchema()

#Convertendo String -> Double


In [ ]:
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql import functions as f

In [ ]:
empresas.printSchema()

In [ ]:
empresas.limit(5).toPandas()

In [ ]:
empresas = empresas.withColumn('capital_social_da_empresa', f.regexp_replace('capital_social_da_empresa', ',','.'))
empresas.limit(5).toPandas()

In [ ]:
empresas = empresas.withColumn('capital_social_da_empresa', empresas['capital_social_da_empresa'].cast(DoubleType()))

In [ ]:
empresas.printSchema()

#Convertendo String -> Date -> Estabelecimentos





In [ ]:
df = spark.createDataFrame([(20200924,), (20201022,), (20210215,)], ['data'])
df.toPandas()
df.printSchema()

In [ ]:
df = df.withColumn("data" , f.to_date(df.data.cast(StringType()), 'yyyyMMdd'))
df.printSchema()

In [ ]:
df.toPandas()

In [ ]:
estabelecimentos.limit(5).toPandas()

In [ ]:
estabelecimentos.printSchema()

In [ ]:
estabelecimentos = estabelecimentos\
  .withColumn(
      "data_situacao_cadastral",
      f.to_date(estabelecimentos.data_situacao_cadastral.cast(StringType()), 'yyyyMMdd')
  )\
  .withColumn(
     "data_de_inicio_atividade",
      f.to_date(estabelecimentos.data_de_inicio_atividade.cast(StringType()), 'yyyyMMdd')
  )\
  .withColumn(
      "data_da_situacao_especial",
      f.to_date(estabelecimentos.data_da_situacao_especial.cast(StringType()), 'yyyyMMdd')
  )

estabelecimentos.printSchema()

In [ ]:
estabelecimentos.limit(5).toPandas()

#Convertendo String -> Date -> Socios

In [ ]:
socios.printSchema()

In [ ]:
socios = socios\
  .withColumn(
      "data_de_entrada_sociedade",
      f.to_date(socios.data_de_entrada_sociedade.cast(StringType()), 'yyyyMMdd')
  )

In [ ]:
socios.limit(5).toPandas()

#Seleções e Consultas

---

#Selecionando Informações => Empresas, Socios e Estabelecimentos

In [ ]:
empresas\
    .select('*')\
    .show(5, False)

In [ ]:
empresas\
    .select('natureza_juridica', 'porte_da_empresa', 'capital_social_da_empresa')\
    .show(5)

In [ ]:
socios\
  .select('nome_do_socio_ou_razao_social', 'faixa_etaria', f.year('data_de_entrada_sociedade').alias('ano_de_entrada'))\
  .show(5,False)

In [ ]:
estabelecimentos\
    .select('nome_fantasia', 'municipio', f.year('data_de_inicio_atividade').alias('ano_de_inicio_atividade'), 
           f.month('data_de_inicio_atividade').alias('mes_de_inicio_atividade'))\
    .show(5)

#Identificando Valores Nulos NaN, Null e none
- Aonde aparecer os dados dos Tipos nulos trocamos para o caracter respectivamente para o seu caracter.

In [ ]:
df = spark.createDataFrame([(1,), (2,), (3,), (None,)], ['data'])
df.toPandas()


In [ ]:
df.show()

In [ ]:
#Com float
df = spark.createDataFrame([(1.,), (2.,), (3.,), (float('nan'),)], ['data'])
df.toPandas()
#NaN é tido como um tipo float

In [ ]:
df.show()

In [ ]:
#Com String
df = spark.createDataFrame([('1',), ('2',), ('3',), (None,)], ['data'])
df.toPandas()

In [ ]:
socios.limit(5).toPandas()

In [ ]:
socios.limit(5).show()

In [ ]:
#Contagem dos valores nulos
socios.select([f.count(f.when(f.isnull(c), 1)).alias(c) for c in socios.columns]).show()


In [ ]:
#Trocando variáveis do tipo numérica (Null) por 0
socios.na.fill(0).limit(5).toPandas()

In [ ]:
#Trocando variáveis do tipo String (NaN) por -
socios.na.fill('-').limit(5).toPandas()

#Ordenando os dados


In [ ]:
#Ordenação em ordem decrescente
socios\
            .select('nome_do_socio_ou_razao_social', 'faixa_etaria', f.year('data_de_entrada_sociedade').alias('ano_de_entrada'))\
            .orderBy('ano_de_entrada', ascending=False)\
            .show(5, False)


In [ ]:
#Ordenando duas colunas 
#A ordem ao passar os valores na lista importa.
socios\
            .select('nome_do_socio_ou_razao_social', 'faixa_etaria', f.year('data_de_entrada_sociedade').alias('ano_de_entrada'))\
            .orderBy(['ano_de_entrada', 'faixa_etaria'], ascending=[True, False])\
            .show(10, False)


#Filtrando os dados

In [ ]:
#filtrar dados da empresa que tem o capital social de 50.0
empresas\
  .where("capital_social_da_empresa==50")\
  .show(5,False)

In [ ]:
#where e filter da para fazer a mesma coisa e aqui posso fazer busca utilizando dois filtros
socios\
  .select("nome_do_socio_ou_razao_social")\
  .filter(socios.nome_do_socio_ou_razao_social.startswith("VINICIUS"))\
  .filter(socios.nome_do_socio_ou_razao_social.endswith("DANTAS"))\
  .limit(10)\
  .toPandas()

#Comando LIKE

In [ ]:
df = spark.createDataFrame([('RESTAURANTE DO RUI',), ('Juca restaurantes ltda',), ('Joca Restaurante',)], ['data'])
df.toPandas()

In [ ]:
#Função que vai colocar todas as letras maiusculas e depois vai trazer todos os restaurentes
df\
  .where(f.upper(df.data).like('%RESTAURANTE%'))\
  .show(truncate=False)

# Se quero filtrar para pesquisa que COMECE com a palavra RESTAURANTE 

df\
  .where(f.upper(df.data).like('RESTAURANTE%'))\
  .show(truncate=False)

# Se quero filtrar para pesquisa que TERMINE com a palavra RESTAURANTE 


df\
  .where(f.upper(df.data).like('%RESTAURANTE'))\
  .show(truncate=False)

In [ ]:
#Aqui eu posso buscar todos os restaurantes que existe na minha tabela
empresas\
    .select('razao_social_nome_empresarial', 'natureza_juridica', 'porte_da_empresa', 'capital_social_da_empresa')\
    .filter(f.upper(empresas['razao_social_nome_empresarial']).like('%RESTAURANTE%'))\
    .show(15, False)

#Agregação e junção


---



#Sumarizando os Dados

In [ ]:
# Um agrupamento com uma contagem dos socios que entraram a partir do ano de 2010
socios\
  .select(f.year('data_de_entrada_sociedade').alias('ano_de_entrada'))\
  .where('ano_de_entrada >= 2010')\
  .groupBy('ano_de_entrada')\
  .count()\
  .orderBy('ano_de_entrada', ascending=True)\
  .show()

In [ ]:
# E feito uma agregação, criando mais de um conjunto de estatísticas dentro do groupBy().
# Utilizando o agg(), aggregate.
#AVG vai retornar a media do capital social da empresa
empresas\
  .select('cnpj_basico', 'porte_da_empresa', 'capital_social_da_empresa')\
  .groupBy('porte_da_empresa')\
  .agg(
      f.avg("capital_social_da_empresa").alias("capital_social_medio"),
      f.count("cnpj_basico").alias("frequencia")
  )\
  .orderBy('porte_da_empresa', ascending=True)\
  .show()

In [ ]:
empresas\
  .select("capital_social_da_empresa")\
  .summary()\
  .show()

  # posso usar qualquer desses metodos dentro do .summary("count", "mean", "stddev", "min", "25%", "50%", "75%", "max"), 
  # e assim ele traria as informações mais filtrada de acordo com a necessidade, o nome do metodo e o describe, ele permite .

#Juntando DataFrame - Joins

#Fazendo a junção dos dados do banco de dados, com JOIN.

In [ ]:
empresas.printSchema()

In [ ]:
socios.printSchema()

In [ ]:
estabelecimentos.printSchema()

In [ ]:
empresas_join = estabelecimentos.join(empresas,'cnpj_basico',how='inner')

In [ ]:
empresas_join.printSchema()
# Como retorno, teremos as informações de ambos os dataframes,
#como porte_da_empresa e capital_social, presentes no conjunto empresas, e nome_fantasia e logradouro, do conjunto estabelecimentos.

In [ ]:
#Nesse dataframe iremos obter nossa tabela de frequencia.
freq = empresas_join\
    .select(
        'cnpj_basico', 
        f.year('data_de_inicio_atividade').alias('data_de_inicio')
    )\
    .where('data_de_inicio >= 2010')\
    .groupBy('data_de_inicio')\
    .agg(f.count("cnpj_basico").alias("frequencia"))\
    .orderBy('data_de_inicio', ascending=True)

In [ ]:
freq.toPandas()


In [ ]:
#A ideia agora e termos uma somatoria da frequencia total ao final desse conjunto.Para isso se usa o metodo union(),
# que e responsavel por juntar dois conjuntos um acima do outro.
#No select farei a criação do rotulo total e a somatórioa das frequencias, e nisso usaremos o f.lit(), uma função que cria o valor literal que 
# chamaremos de Total.
#E depois chama f.sum() para ser somado toda a colunda freq.frequencia, e depois renomearemos como frequencia.
freq.union(
    freq.select(
        f.lit('Total').alias('data_de_inicio'),
        f.sum(freq.frequencia).alias('frequencia')    
        )
).show()

#SparkSQL

In [ ]:
empresas.createOrReplaceTempView("empresasView")

In [ ]:
spark.sql("SELECT * FROM empresaView").show(5)

In [ ]:
spark\
  .sql("""
    SELECT *
        FROM empresasView
        WHERE capital_social_da_empresa = 50

  """)\
  .show(5)

In [ ]:
#Pegando a media das empresas
spark\
    .sql("""
        SELECT porte_da_empresa, MEAN(capital_social_da_empresa) AS Media 
            FROM empresasView 
            GROUP BY porte_da_empresa
    """)\
    .show(5)



In [ ]:
empresas_join.createOrReplaceTempView("empresasJoinView")

In [ ]:
freq = spark\
    .sql("""
        SELECT YEAR(data_de_inicio_atividade) AS data_de_inicio, COUNT(cnpj_basico) AS count
            FROM empresasJoinView 
            WHERE YEAR(data_de_inicio_atividade) >= 2010
            GROUP BY data_de_inicio
            ORDER BY data_de_inicio
    """)

freq\
    .show()

In [ ]:
freq.createOrReplaceTempView("freqView")

In [ ]:
spark\
    .sql("""
        SELECT *
            FROM freqView
        UNION ALL
        SELECT 'Total' AS data_de_inicio, SUM(count) AS count
            FROM freqView
    """)\
    .show()

#Arquivos CSV

In [ ]:
#armazenando o arquivo
empresas.write.csv(
    path= '/content/drive/MyDrive/curso-spark/empresas/csv',
    mode='overwrite',
    sep=';',
    header=True
)


In [ ]:
#acessando como fico salvo
empresas2= spark.read.csv(
    path= '/content/drive/MyDrive/curso-spark/empresas/csv',
    sep=';',
    inferSchema=True,
    header=True
)


In [ ]:
empresas2.printSchema()

In [ ]:
socios.write.csv(
    path='/content/drive/MyDrive/curso-spark/socios/csv',
    mode='overwrite',
    sep=';',
    header=True
)

In [ ]:
socios2 = spark.read.csv(
    path='/content/drive/MyDrive/curso-spark/socios/csv',
    sep=';',
    inferSchema=True,
    header=True
)

In [ ]:
socios2.printSchema()

In [ ]:
estabelecimentos.write.csv(
    path='/content/drive/MyDrive/curso-spark/estabelecimentos/csv',
    mode='overwrite',
    sep=';',
    header=True
)

In [ ]:
estabelecimentos2 = spark.read.csv(
    path='/content/drive/MyDrive/curso-spark/estabelecimentos/csv',
    sep=';',
    inferSchema=True,
    header=True

)

In [ ]:
estabelecimentos2.printSchema()